In [9]:
from dotenv import load_dotenv
import os

# env variables
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Data Ingestion(Load Source data)


In [67]:
from langchain_community.document_loaders import TextLoader

tloader = TextLoader("speech.txt")
text_documents = tloader.load()
text_documents

[Document(page_content='Main menu WikipediaThe Free Encyclopedia Search Create account Log in Personal tools Toggle the table of contents Text Article Talk Read Edit View history Tools From Wikipedia, the free encyclopedia For the Wikipedia help page, see Help:Wikitext. Look up Text, Texts, text, or texts in Wiktionary, the free dictionary. Text may refer to: Written word Text (literary theory), any object that can be read, including: Religious text, a writing that a religious tradition considers to be sacred Text, a verse or passage from scripture used in expository preaching Textbook, a book of instruction in any branch of study Computing and telecommunications Plain text, unformatted text Text file, a type of computer file opened by most text software Text string, a sequence of characters manipulated by software Text message, a short electronic message designed for communication between mobile phone users Text (Chrome app), a text editor for the Google Chrome web browser tEXt, an an

### Reading from text web


In [68]:
from langchain_community.document_loaders import WebBaseLoader
import bs4
import json


#### load, chunk  and index the content

wloader = WebBaseLoader(
    web_paths=[
        "http://lilianweng.github.io/posts/2023-06-23-agent/",
        "https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.web_base.WebBaseLoader.html",
    ],
    bs_kwargs=dict(
        # parse_only=bs4.SoupStrainer(
        #     class_=("post-title", "post-content", "post-header")
        # )  # classes to read
    ),
)

text_documents = wloader.load()
[i for i, v in dict(text_documents[0]).items()]

['page_content', 'metadata', 'type']

In [69]:
text_documents[0].metadata

{'source': 'http://lilianweng.github.io/posts/2023-06-23-agent/',
 'title': "LLM Powered Autonomous Agents | Lil'Log",
 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:',
 'language': 'en'}

In [70]:
for key in dict(text_documents[0]):
    print(key)

page_content
metadata
type


In [71]:
text_documents[1]

Document(page_content="\n\n\n  \n\n\n\n\nlangchain_community.document_loaders.web_base.WebBaseLoader — ðŸ¦œðŸ”— LangChain 0.1.16\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLangChain\n\n\nCore\n\n\nCommunity\n\n\nExperimental\n\n\nText splitters\n\n\nai21\n\n\nairbyte\n\n\nanthropic\n\n\nastradb\n\n\nchroma\n\n\ncohere\n\n\nelasticsearch\n\n\nexa\n\n\nfireworks\n\n\ngoogle-genai\n\n\ngoogle-vertexai\n\n\ngroq\n\n\nibm\n\n\nmistralai\n\n\nmongodb\n\n\nnomic\n\n\nnvidia-ai-endpoints\n\n\nnvidia-trt\n\n\nopenai\n\n\npinecone\n\n\npostgres\n\n\nrobocorp\n\n\ntogether\n\n\nupstage\n\n\nvoyageai\n\n\nPartner libs\n\nai21\nairbyte\nanthropic\nastradb\nchroma\ncohere\nelasticsearch\nexa\nfireworks\ngoogle-genai\ngoogle-vertexai\ngroq\nibm\nmistralai\nmongodb\nnomic\nnvidia-ai-endpoints\nnvidia-trt\nopenai\npinecone\npostgres\nrobocorp\ntogether\nupstage\nvoyageai\n\n\n\nDocs\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nToggle Menu\n\n\n\nPrev\nUp\nNext\n\n\n\nlangchain_community.document_loaders.web_base.W

## Reading from PDF


In [72]:
from langchain_community.document_loaders import PyPDFLoader

ploader = PyPDFLoader(
    "/home/zohaib/langchainKnaik/PDFs/Graph Representation Learning.pdf"
)
pages = ploader.load()
len(pages)

141

# Transform after loading


In [74]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
)

docs = text_splitter.split_documents(pages)
docs

[Document(page_content='Graph Representation Learning\nWilliam L. Hamilton\nMcGill University\n2020', metadata={'source': '/home/zohaib/langchainKnaik/PDFs/Graph Representation Learning.pdf', 'page': 0}),
 Document(page_content='2020\nPre-publication draft of a book to be published by\nMorgan & Claypool publishers.', metadata={'source': '/home/zohaib/langchainKnaik/PDFs/Graph Representation Learning.pdf', 'page': 0}),
 Document(page_content='Unedited version released with permission.\nAll relevant copyrights held by the author and', metadata={'source': '/home/zohaib/langchainKnaik/PDFs/Graph Representation Learning.pdf', 'page': 0}),
 Document(page_content='publisher extend to this pre-publication draft.', metadata={'source': '/home/zohaib/langchainKnaik/PDFs/Graph Representation Learning.pdf', 'page': 0}),
 Document(page_content='Citation: William L. Hamilton. (2020). Graph Representation Learning.', metadata={'source': '/home/zohaib/langchainKnaik/PDFs/Graph Representation Learning.p

## converting chunks to embeddings


In [76]:
# vector mebedding and vecyor storer
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

db = Chroma.from_documents(docs[:20], OpenAIEmbeddings())
db

In [77]:
query = "What is this about?"
docs = db.similarity_search(query)
docs

[Document(page_content='2020\nPre-publication draft of a book to be published by\nMorgan & Claypool publishers.', metadata={'page': 0, 'source': '/home/zohaib/langchainKnaik/PDFs/Graph Representation Learning.pdf'}),
 Document(page_content='2020\nPre-publication draft of a book to be published by\nMorgan & Claypool publishers.', metadata={'page': 0, 'source': '/home/zohaib/langchainKnaik/PDFs/Graph Representation Learning.pdf'}),
 Document(page_content='publisher extend to this pre-publication draft.', metadata={'page': 0, 'source': '/home/zohaib/langchainKnaik/PDFs/Graph Representation Learning.pdf'}),
 Document(page_content='publisher extend to this pre-publication draft.', metadata={'page': 0, 'source': '/home/zohaib/langchainKnaik/PDFs/Graph Representation Learning.pdf'})]

In [82]:
xdb = Chroma.from_texts(
    texts=[x.page_content for x in docs[:20]], embedding=OpenAIEmbeddings()
)
xdocs = xdb.similarity_search(query)
xdocs

[Document(page_content='2020\nPre-publication draft of a book to be published by\nMorgan & Claypool publishers.', metadata={'page': 0, 'source': '/home/zohaib/langchainKnaik/PDFs/Graph Representation Learning.pdf'}),
 Document(page_content='2020\nPre-publication draft of a book to be published by\nMorgan & Claypool publishers.', metadata={'page': 0, 'source': '/home/zohaib/langchainKnaik/PDFs/Graph Representation Learning.pdf'}),
 Document(page_content='2020\nPre-publication draft of a book to be published by\nMorgan & Claypool publishers.'),
 Document(page_content='2020\nPre-publication draft of a book to be published by\nMorgan & Claypool publishers.')]

In [83]:
# $ using Faiss vector db
from langchain_community.vectorstores import FAISS

fdb = FAISS.from_texts(
    texts=[x.page_content for x in docs[:20]], embedding=OpenAIEmbeddings()
)
fdocs = xdb.similarity_search(query)
fdocs

[Document(page_content='2020\nPre-publication draft of a book to be published by\nMorgan & Claypool publishers.', metadata={'page': 0, 'source': '/home/zohaib/langchainKnaik/PDFs/Graph Representation Learning.pdf'}),
 Document(page_content='2020\nPre-publication draft of a book to be published by\nMorgan & Claypool publishers.', metadata={'page': 0, 'source': '/home/zohaib/langchainKnaik/PDFs/Graph Representation Learning.pdf'}),
 Document(page_content='2020\nPre-publication draft of a book to be published by\nMorgan & Claypool publishers.'),
 Document(page_content='2020\nPre-publication draft of a book to be published by\nMorgan & Claypool publishers.')]